# Exercise 4 - Ensemble of Batman trees
## Imports

In [64]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
import pickle
import os
print("Current working directory:", os.getcwd(), sep="\n")

# So that changes to the a3 model are reflected here.
import a3
import importlib
importlib.reload(a3)
import a3


Current working directory:
/Users/atakancoban/Desktop/School/2dv506 - Machine learning/Assignment 3/Lecture_9


## Functions

In [65]:
def nonce():
    pass

## Part 1 - Prepare datasets

In [66]:
# Load bm.csv.
data = np.loadtxt("datasets/bm.csv", delimiter=',')
np.random.shuffle(data)
X_train, y_train = data[:5000, 0:2], data[:5000, 2]
X_test, y_test = data[5000:, 0:2], data[5000:, 2]

# Bootstrap 100 unique datasets.
rng = np.random.default_rng()

n = 5000
#r = np.zeros([n, 100], dtype=int)  # TODO: Delete if not used
XX = np.zeros([n, 2, 100])
yy = np.zeros([n, 100], dtype=int)

for i in range(100):
    #r[:, i] = rng.choice(n, size=n, replace=True)  # TODO: Delete if not used
    idxs = rng.choice(n, size=n, replace=True)
    XX[:, :, i] = X_train[idxs, :]
    yy[:, i] = y_train[idxs]

## Part 2 - Train model and calculate performance

In [67]:
# Train model.
clfs = []
for i in range(100):
    clfs.append(DecisionTreeClassifier().fit(XX[:,:,i], yy[:,i]))

# Evaluation.
def test_accuracy(X_test, y_test, y_test_pred):
    y_diff = y_test - y_test_pred
    y_diff = (y_diff**2)**0.5
    error_count = np.sum(y_diff)
    return (X_test.shape[0] - error_count) / X_test.shape[0]

# Ensemble accuracy.
y_test_pred = a3.ensemble_predict_binary(clfs, X_test)
accuracy = round(test_accuracy(X_test, y_test, y_test_pred)*100, 2)
print(f"Ensemble accuracy: {accuracy}%")

# Average individual decision tree accuracy.
total = 0
for clf in clfs:
    y_test_pred = clf.predict(X_test)
    total += test_accuracy(X_test, y_test, y_test_pred)
accuracy = round((total/len(clfs))*100, 2)
print(f"Average individual decision tree accuracy: {accuracy}%")

Ensemble accuracy: 98.24%
Average individual decision tree accuracy: 96.74%


## Part 3 - Prepare decision boundaries for individual classifiers

In [ ]:
# Takes a couple minutes.
boundaries = []

# Prepare decision boundaries.
for plot in range(100):
    print(f"Calculating boundaries for classifier {plot}")

    # Prepare meshgrid.
    x_min, x_max, x_step = -9, 9, 0.05
    y_min, y_max, y_step = -5, 5, 0.05
    xx, yy = np.meshgrid(np.arange(x_min, x_max, x_step), np.arange(y_min, y_max, y_step))
    zz = np.zeros(xx.shape)

    # Predict for each point in meshgrid.
    for i in range(zz.shape[0]):
        for j in range(zz.shape[1]):
            test = np.array([[xx[i,j], yy[i,j]]])
            zz[i, j] = clfs[plot].predict(test)

    # Save boundary points.
    boundaries.append((xx, yy, zz))

Calculating boundaries for classifier 0
Calculating boundaries for classifier 1
Calculating boundaries for classifier 2
Calculating boundaries for classifier 3
Calculating boundaries for classifier 4
Calculating boundaries for classifier 5
Calculating boundaries for classifier 6
Calculating boundaries for classifier 7
Calculating boundaries for classifier 8
Calculating boundaries for classifier 9
Calculating boundaries for classifier 10
Calculating boundaries for classifier 11
Calculating boundaries for classifier 12
Calculating boundaries for classifier 13
Calculating boundaries for classifier 14
Calculating boundaries for classifier 15
Calculating boundaries for classifier 16
Calculating boundaries for classifier 17
Calculating boundaries for classifier 18
Calculating boundaries for classifier 19
Calculating boundaries for classifier 20
Calculating boundaries for classifier 21
Calculating boundaries for classifier 22
Calculating boundaries for classifier 23
Calculating boundaries for

## Part 4 - Plot decision boundaries for individual classifiers

In [ ]:
fig = plt.figure(figsize=(20,20))

# Plot for each classifier.
for i in range(100):
    print(f"Plotting classifier #{i}")

    xx, yy, zz = boundaries[i]
    ax = fig.add_subplot(10, 10, i+1)
    ax.contour(xx, yy, zz, alpha=0.4, cmap="Reds")

## Part 5 - Prepare decision boundaries for the ensemble

## Part 6 - Plot decision boundaries for the ensemble